In [ ]:

import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import Select #下拉式選單
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import requests
import time
from datetime import datetime
import ddddocr
import os, io
from PIL import Image
import json

In [ ]:
file_path = ' '
with open ( f"{file_path}/config.json", "r" ) as r:
    config = json.load(r)

## 指定變數訂票資訊
# 出發站
StartStation_name = config['StartStation_name']
# 抵達站
DestinationStation_name = config['DestinationStation_name']
# 出發日期
Date = config['Date']
# 幾點以後的班次
Dep_time = config['Dep_time']
# 全票數
ticket_num = config['ticket_num']

### 訂票資訊 ###
PersonID = config['PersonID'] #輸入身份證碼
PhoneNum = config['PhoneNum'] #輸入電話號碼


In [ ]:
# 轉換成中文月份
def chinese_date_format( date ):
    chinese_dict = {
        1:'一月',
        2:'二月',
        3:'三月',
        4:'四月',
        5:'五月',
        6:'六月',
        7:'七月',
        8:'八月',
        9:'九月',
        10:'十月',
        11:'十一月',
        12:'十二月',   
    }
    year = date.year
    month = date.month
    day = date.day
    return '{} {}, {}'.format(chinese_dict[month], day, year)

In [ ]:
# 消除webdriver
opt = webdriver.ChromeOptions()
opt.add_experimental_option('excludeSwitches',['enable-automation'])
opt.add_experimental_option('useAutomationExtension',False)

# 告訴Chrome去除webdriver
opt.add_argument('disable-blink-features=AutomationControlled')

# 進入網頁
driver = webdriver.Chrome(options=opt)
url = "https://irs.thsrc.com.tw/IMINT/?locale=tw" #高鐵訂票網站
driver.get(url)

# pop up 一開始進入網站的cookie
time.sleep(2)
driver.find_element("id", 'cookieAccpetBtn').click() #點擊初始cookie

### 建立站名對應值 ###
time.sleep(1)
Station_dict = {}
StartStation = Select(driver.find_element(By.NAME, "selectStartStation")) #選擇下拉式選單內容
for op in StartStation.options:
    # print(op.get_dom_attribute("value"), op.text)
    Station_dict [op.text] = op.get_dom_attribute("value")
# print(Station_dict)

### 選擇出發站 ###
time.sleep(1)
StartStation = Select(driver.find_element(By.NAME, "selectStartStation")) #選擇下拉式選單內容
StartStation_value = Station_dict.get(StartStation_name)
StartStation.select_by_value(StartStation_value)
print('出發站', StartStation_name , StartStation_value )

### 選擇抵達站 ###
time.sleep(1)
DestinationStation = Select(driver.find_element(By.NAME, "selectDestinationStation")) #選擇下拉式選單內容
DestinationStation_value = Station_dict.get(DestinationStation_name)
DestinationStation.select_by_value(DestinationStation_value)
print('抵達站', DestinationStation_name,DestinationStation_value)

### 選擇日期 ###
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, "[data-triptype='oneway']").click() #找到選擇的按鈕
StartDay = driver.find_element(By.CLASS_NAME, "flatpickr-days").find_elements(By.CSS_SELECTOR, ".flatpickr-day:not(.flatpickr-disabled):not(.hidden)")
# for i in StartDay:
#     print(i.get_dom_attribute('aria-label') ) #找出可選擇的日期

#轉換預計出發日期
Dep_Date = datetime.strptime(Date, '%Y/%m/%d')
chinese_date = chinese_date_format(Dep_Date)
print(chinese_date)

driver.find_element(By.CSS_SELECTOR, "[aria-label="+ "'" + chinese_date+"']").click() # 選擇出發日期

#篩選出發時間
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, "[data-triptype='oneway']").click()
DepartTime = Select(driver.find_element(By.NAME, "toTimeTable"))
DepartTime.select_by_visible_text(Dep_time)
print('Dep_time is done')

### 選擇票數 ###
# 全票
time.sleep(1)
Ticket = Select(driver.find_element(By.NAME, "ticketPanel:rows:0:ticketAmount"))
Ticket.select_by_visible_text(ticket_num)
print('ticket_num is done')

### 處理驗證碼 ###
# 圖片存放位置
img_path = ' ' 
#根據驗證碼id 截圖(screenshot_as_png)
img = driver.find_element(By.ID, "BookingS1Form_homeCaptcha_passCode").screenshot_as_png  
# 將截取的圖片保存到指定路徑
with open(f"{img_path}/captcha.png", "wb") as img_file:
    img_file.write(img)
# 使用PIL打開圖片
imageStream = io.BytesIO(img) #用io套件轉為二進制 暫存
im = Image.open(imageStream)
# 解讀驗證碼
ocr = ddddocr.DdddOcr()
with open( f"{img_path}/captcha.png" , 'rb') as f:
    img_bytes = f.read()
VERIFY = ocr.classification(img_bytes)
# print(type(VERIFY))
print('securityCode',VERIFY)

# 填入驗證碼
driver.find_element(By.NAME,"homeCaptcha:securityCode").send_keys(VERIFY)

# 開始查詢
driver.find_element(By.ID,"SubmitButton").click()

# 當出現驗證碼錯誤的時候(整理驗證碼&選擇票數及種類)
t=1
while driver.find_element(By.CLASS_NAME,"alert-content") and ( 'https://irs.thsrc.com.tw/IMINT/?wicket:interface=:0' in driver.current_url ) :
    time.sleep(1)
    print('try error', t)
    # 清除驗證碼
    driver.find_element(By.NAME,"homeCaptcha:securityCode").clear()
    # 重新整理圖片
    driver.find_element(By.CLASS_NAME,"material-icons").click()

    #篩選出去程時間
    time.sleep(2)
    driver.find_element(By.CSS_SELECTOR, "[data-triptype='oneway']").click()
    DepartTime = Select(driver.find_element(By.NAME, "toTimeTable"))
    DepartTime.select_by_visible_text(Dep_time)


    #選擇票數
    # 全票
    time.sleep(1)
    Ticket = Select(driver.find_element(By.NAME, "ticketPanel:rows:0:ticketAmount"))
    Ticket.select_by_visible_text(ticket_num)

    #根據驗證碼id 截圖(screenshot_as_png)
    img = driver.find_element(By.ID, "BookingS1Form_homeCaptcha_passCode").screenshot_as_png  

    # 將截取的圖片保存到指定路徑
    with open(f"{img_path}/captcha.png", "wb") as img_file:
        img_file.write(img)

    # 使用PIL打開圖片
    imageStream = io.BytesIO(img) #用io套件轉為二進制 暫存
    im = Image.open(imageStream)

    # 解讀驗證碼
    ocr = ddddocr.DdddOcr()
    with open( f"{img_path}/captcha.png" , 'rb') as f:
        img_bytes = f.read()
    VERIFY = ocr.classification(img_bytes)
    print(type(VERIFY))
    print('securityCode',VERIFY)

    # 填入驗證碼
    driver.find_element(By.NAME,"homeCaptcha:securityCode").send_keys(VERIFY)

    # 開始查詢
    driver.find_element(By.ID,"SubmitButton").click()
    t += 1

### 選擇車次 ###
# 點擊出現的第一個班次
time.sleep(1)
driver.find_element(By.CLASS_NAME, "result-item").click()
# click "確認車次"
driver.find_element(By.NAME, "SubmitButton").click()

### 輸入訂票資訊 ###
time.sleep(1)
driver.find_element(By.ID,"idNumber").send_keys( PersonID )
driver.find_element(By.ID,"mobilePhone").send_keys( PhoneNum )

# 非會員
driver.find_element(By.ID,"memberSystemRadio3").click()
# 勾選同意
driver.find_element(By.NAME,"agree").click()
# 完成訂位
driver.find_element(By.ID,"isSubmit").click()

### 留存訂位代碼 ###
file_path = '/Users/sandy/Python/commeet/file'
ticket_Code = driver.find_element(By.CLASS_NAME, 'pnr-code').text
with open( f'{file_path}/ticket_Code.log', 'a') as a_writer:
    a_writer.write( ticket_Code + ' ' + str(datetime.now()) +'\n')
print('ticket_Code get', ticket_Code)

# driver.quit()

In [ ]:
driver.quit()
# driver.refresh()